In [1]:
import numpy as np
import cv2

In [2]:
# Method 1: Using Lucas-Kanade method for sparse density of optical flow
corner_track_params = dict(maxCorners = 10,qualityLevel=0.3, minDistance=7,blockSize=7)

In [3]:
# When using Lucas-Kanade window size is important (small = sensitive to noise, big = might not be sensitive to small movements)
# More iterations (in this case 10), will result in more exhaustive search
# Smaller epsiolon = finish earlier
# So we are exchanging speed with accuracy with these to params
lk_params = dict(winSize=(200,200), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [4]:
# Saving initial frame
cap = cv2.VideoCapture(0)
ret, prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

In [5]:
# Points to tracks (maxCorners we want to track are 10)
prevPts = cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params)

# Mask to draw lines into the video (for displaying purposes)
mask = np.zeros_like(prev_frame)

while True:
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    nextPts,status,err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPts,None,**lk_params)
#     Status == 1 means the flow for corresponding feature has been found
    good_new = nextPts[status == 1]
    good_prev = prevPts[status == 1]
    
#     Iterating through the good new points and the previosuly good ones (those where optical flow was detected)
    for i, (new,prev) in enumerate(zip(good_new,good_prev)):
        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()
        
#         Draw tracking line of the points as they change from frame to frame
        mask = cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)
        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
    
#     Adding the mask that has the lines and the frame that has the circles to new image
    img = cv2.add(frame,mask)
    cv2.imshow('tracking', img)
    
    k = cv2.waitKey(30) & 0xFF 
    if k == 27:
        break
    
#     Before moving to new frame, updating points values
    prev_gray = frame_gray.copy()
#     This is the shape that the Lucas-Kanade returns
    prevPts = good_new.reshape(-1,1,2)
    
    
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [ ]:
# Method 2: Using Gunner Farneback's algorithm for dense optical flow

In [1]:
import numpy as np
import cv2

In [2]:
cap = cv2.VideoCapture(0)
ret, frame1 = cap.read()
prvsImg = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Make a hsv mask with max saturation
hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

# If movement is to the left, it will color it blue
# If it to the right, it will color it red
# Up is purple
# Down is green
# No movement is black
while True:
    ret, frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg,None,0.5,3,15,3,5,1.2,0)
#     Convert flow into polar coordinates (so that we can map the resulting vector into the HSV Map)
#     For every pixel in flow, grab the horizontal coordinates, and then the vertical ones
#     Then convert them to magnitude and angle
#     This way we can color the frame using the hue of the map (which depends on magnitude and angle)
    mag, ang = cv2.cartToPolar(flow[:,:,0],flow[:,:,1], angleInDegrees= True)
#     Change the hue to angle (/2 so we can scale it using 360 to 180)
    hsv_mask[:,:,0] = ang/2
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    
    cv2.imshow('frame', bgr)
    k = cv2.waitKey(10) & 0xFF 
    if k == 27:
        break
        
#     Update for next frame
    prvsImg = nextImg

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1